In [1]:
from qiskit import ClassicalRegister, QuantumRegister
from qiskit import QuantumCircuit, execute
from qiskit import register, available_backends, get_backend

#import Qconfig and set APIToken and API url
import sys
sys.path.append("../../") # go to parent dir
import Qconfig
qx_config = {
    "APItoken": Qconfig.APItoken,
    "url": Qconfig.config['url']}

#set api
register(qx_config['APItoken'], qx_config['url'])

The following defines and runs an implementation of the $d=8$ repetition code with a stored `0`.

In [2]:
num = 16
device = 'ibmqx5'
shots = 8192

In [3]:
script = []
for test in range(3):
    
    q = QuantumRegister(num)
    c = ClassicalRegister(num)
    script.append( QuantumCircuit(q, c) )

    script[test].barrier()

    script[test].cx(q[6],q[5])
    script[test].h(q[4])
    script[test].h(q[3])
    script[test].cx(q[3],q[4])
    script[test].h(q[4])
    script[test].h(q[3])
    if test in [1,2]:
        script[test].h(q[2])
        script[test].h(q[1])
        script[test].cx(q[1],q[2])
        script[test].h(q[2])
        script[test].h(q[1])
    if test in [2]:
        script[test].h(q[0])
        script[test].h(q[15])
        script[test].cx(q[15],q[0])
        script[test].h(q[0])
        script[test].h(q[15])

    script[test].barrier()
    
    script[test].h(q[4])
    script[test].h(q[5])
    script[test].cx(q[5],q[4])
    script[test].h(q[4])
    script[test].h(q[5])
    script[test].cx(q[2],q[3])
    if test in [1,2]:
        script[test].h(q[0])
        script[test].h(q[1])
        script[test].cx(q[1],q[0])
        script[test].h(q[0])
        script[test].h(q[1])
    if test in [2]:
        script[test].h(q[14])
        script[test].h(q[15])
        script[test].cx(q[15],q[14])
        script[test].h(q[14])
        script[test].h(q[15])

    script[test].barrier()

    for address in range(num):
        script[test].measure(q[address], c[address])

backend = get_backend(device)
print('Status of '+device+':',backend.status)
        
job = execute(script, backend, shots=shots, skip_translation=True)

Status of ibmqx5: {'available': True, 'pending_jobs': 33, 'name': 'ibmqx5'}


In [4]:
results = []
for test in range(3):
    results.append( job.result().get_counts(script[test]) )

The probability that each qubit is output with value `1` is determined and printed. The [standard qubit numbering](https://github.com/QISKit/ibmqx-backend-information/blob/master/backends/ibmqx5/README.md) is used. To flag up the higher probabilities, they are printed further toward the right.

In [5]:
for test in range(3):
    if test==0:
        print("\nResults for d=3 repetition code on qubits q1 to q6\n")
    elif test==1:
        print("\nResults for d=4 repetition code on qubits q0 to q6\n")
    else:
        print("\nResults for d=5 repetition code on qubits q0 to q6, q14 and q15\n")
    
    p = [0]*num
    for qubit in range(num):
        for string in results[test].keys():
            if string[num-1-qubit]=='1':
                p[qubit] += results[test][string]/shots

        print( "Q" + str(qubit) + " "*(qubit<10) + "  " + " "*int(15*p[qubit]) + str(p[qubit]) )


Results for d=3 repetition code on qubits q1 to q6

Q0   0.0145263671875
Q1   0.0118408203125
Q2   0.0552978515625
Q3    0.087646484375
Q4   0.06494140625
Q5   0.0606689453125
Q6   0.0301513671875
Q7   0.00146484375
Q8   0.003662109375
Q9    0.1021728515625
Q10  0.052978515625
Q11  0.0140380859375
Q12   0.0946044921875
Q13  0.0052490234375
Q14  0.042236328125
Q15  0.0177001953125

Results for d=4 repetition code on qubits q0 to q6

Q0   0.0338134765625
Q1          0.4747314453125
Q2   0.0560302734375
Q3    0.095703125
Q4    0.07177734375
Q5    0.0684814453125
Q6   0.0302734375
Q7   0.0008544921875
Q8   0.0020751953125
Q9    0.1234130859375
Q10  0.049072265625
Q11  0.0133056640625
Q12   0.0870361328125
Q13  0.0076904296875
Q14  0.0400390625
Q15  0.0179443359375

Results for d=5 repetition code on qubits q0 to q6, q14 and q15

Q0   0.052001953125
Q1          0.48046875
Q2   0.055908203125
Q3    0.1099853515625
Q4    0.0772705078125
Q5   0.064697265625
Q6   0.0283203125
Q7   0.01171875
Q